# What is Parkinson Disease
Parkinson’s disease is a nervous system disorder that affects movement. If you have seen the movie Love & other drugs then you know something about it. There are various stages in Parkinson Disease. <br>
Stage 1: Mild symptoms that do not typically interfere with daily life, including tremors and movement issues on only one side of the body.<br>
Stage 2: Symptoms continue to become worse with both tremors and rigidity now affecting both sides of the body. Daily tasks become challenging.<br>
Stage 3: Loss of balance and movements with falls becoming frequent and common. The patient is still capable of (typically) living independently.<br>
Stage 4: Symptoms become severe and constraining. The patient is unable to live alone and requires help to perform daily activities.<br>
Stage 5: Likely impossible to walk or stand. The patient is most likely wheelchair bound and may even experience hallucinations.<br>

Detection Parkinson Disease involves analysing the Spiral and Wave pattern drawn by the patients. <br>
Dataset : http://www.niats.feelt.ufu.br/en/node/81

Spiral: 102 images, 72 training, and 30 testing<br>
Wave: 102 images, 72 training, and 30 testing